# Домашнее задание 2 по обработке текстов

Рассмотрим задачу бинарной классификации. Пусть дано два списка имен: мужские и женские имена. Требуется разработать классификатор, который по данному имени будет определять мужское оно или женское.

Данные: 
* Женские имена: female.txt
* Мужские имена: male.txt

## Часть 1. Предварительная обработка данных

1. Удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими дновременно), если такие есть; 
2. Создайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы, т.е. к классу принадлежало бы одинаковое количество имен;

In [88]:
import pandas as pd
male = pd.read_csv('male.txt', header=None, names = ["names"])
female = pd.read_csv('female.txt', header=None, names = ["names"])
male["sex"] = 1
female["sex"] = 0
tmp = male.copy()
tmp = tmp.append(female, ignore_index=True)
tmp.count()


names    7944
sex      7944
dtype: int64

Удалим повторяющиеся имена:

In [89]:
tmp.drop_duplicates('names',keep=False,inplace=True) 
testPart = 0.1 #Доля тестовой быборки
testPart = int(testPart * tmp.count()[0] / 2) #Рассчитаем так, чтобы число мужчин и женщин было равно


In [90]:
X_test = tmp[:testPart]
X_test = X_test.append(tmp[-testPart:])
X_test.count()[0]

720

In [91]:
X_train = tmp[testPart:-testPart]
X_train.count()[0]

6494

Проверим правильность деления

In [92]:
(tmp.count()[0] == (X_test.count()[0] + X_train.count()[0]))

True

Отделим y_test и y_train и удалим их из X_test и X_train

In [93]:
y_test = X_test["sex"]
y_train = X_train["sex"]
X_test.drop('sex', axis=1);
X_train.drop('sex', axis=1);

##  Часть 2. Базовый метод классификации

Используйте метод наивного Байеса или логистическую регрессию для классификации имен: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

Для генерации $n$-грамм используйте:

In [98]:
from nltk.util import ngrams

##  Часть 3. Нейронная сеть


Используйте  реккурентную нейронную сеть с  LSTM для решения задачи. В ней может быть несколько слоев с LSTM, несколько слоев c Bidirectional(LSTM).  У нейронной сети один выход, определяющий класс имени. 

Представление имени для классификации в этом случае: бинарная матрица размера (количество букв в алфавите $\times$ максимальная длина имени). Обозначим его через $x$. Если первая буква имени a, то $x[1][1] = 1$, если вторая – b, то  $x[2][1] = 1$.  

Не забудьте про регуляризацию нейронной сети дропаутами. 

Сравните результаты классификации разными методами. Какой метод лучше и почему?

Сравните результаты, получаемые при разных значениях дропаута, разных числах узлов на слоях нейронной сети по $F$-мере и аккуратности. В каких случаях нейронная сеть ошибается?

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras import __version__ as keras_version
import numpy as np


nEpochs = 10
weightsFileName = "gender_weights.h5"


print ("loading data")

with open("male.txt") as f:
    m_names = f.readlines()

with open("female.txt") as f:
    f_names = f.readlines()

mf_names = []

for f_name in f_names:
	if f_name in m_names:
		mf_names.append(f_name)

m_names = [m_name.lower() for m_name in m_names if not m_name in mf_names]
f_names = [f_name.lower() for f_name in f_names if not f_name in mf_names]


totalEntries = len(m_names) + len(f_names)
maxlen = len(max( m_names , key=len)) + len(max( f_names , key=len))

chars = set(  "".join(m_names) + "".join(f_names)  )
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


print ("total endtries " , totalEntries)
print ("max len " , maxlen)
print('total chars:', len(chars))



X = np.zeros((totalEntries , maxlen, len(chars) ), dtype=np.bool)
y = np.zeros((totalEntries , 2 ), dtype=np.bool)


for i, name in enumerate(m_names):
    for t, char in enumerate(name):
        X[i, t, char_indices[char]] = 1
    y[i, 0 ] = 1

for i, name in enumerate(f_names):
    for t, char in enumerate(name):
        X[i + len(m_names), t, char_indices[char]] = 1
    y[i + len(m_names) , 1 ] = 1

def vec2c(vec):
	for i,v in enumerate(vec):
		if v:
			return indices_char[i]
	return ""

print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop')


json_string = model.to_json()

with open("model.json", "w") as text_file:
    text_file.write(json_string)


if keras_version[0] == '1':
	model.fit(X, y, batch_size=16, nb_epoch=nEpochs)
else:
	model.fit(X, y, batch_size=16, epochs=nEpochs)

model.save_weights('my_model_weights.h5')

print ("done and weights saved")
score = model.evaluate(X, y, batch_size=16)
print ("score " , score)


Using TensorFlow backend.


loading data
total endtries  7214
max len  32
total chars: 30
Build model...
Epoch 1/10
7214/7214 [==============================] - 297s 41ms/step - loss: 0.7138
Epoch 2/10
7214/7214 [==============================] - 308s 43ms/step - loss: 0.6652
Epoch 3/10
 320/7214 [>.............................] - ETA: 4:38 - loss: 0.6545

In [ ]:
from keras.models import model_from_json
import numpy as np
import sys


def predict(model, name, maxlen, chars, char_indices):
	x = np.zeros((1, maxlen, len(chars)))
	for t, char in enumerate(name):
		x[0, t, char_indices[char]] = 1

	preds = model.predict(x, verbose=0)[0]
	return preds


def load_model(maleFile, femaleFile):
	mf_names = []
	with open(maleFile) as f:
		m_names = f.readlines()

	with open(femaleFile) as f:
		f_names = f.readlines()

	for f_name in f_names:
		if f_name in m_names:
			mf_names.append(f_name)

	m_names = [m_name.lower() for m_name in m_names if not m_name in mf_names]
	f_names = [f_name.lower() for f_name in f_names if not f_name in mf_names]

	totalEntries = len(m_names) + len(f_names)
	maxlen = len(max( m_names , key=len)) + len(max( f_names , key=len))

	chars = set(  "".join(m_names) + "".join(f_names)  )
	char_indices = dict((c, i) for i, c in enumerate(chars))
	indices_char = dict((i, c) for i, c in enumerate(chars))

	with open("model.json", 'r') as content_file:
		json_string = content_file.read()

	model = model_from_json(json_string)
	model.load_weights('my_model_weights.h5')
	return model, maxlen ,chars ,char_indices


if __name__ == "__main__":
	model, maxlen, chars, char_indices = load_model(sys.argv[1], sys.argv[2])
	while True:
		(print "Enter any name:")
		n = raw_input()
		v = predict(model, n, maxlen, chars, char_indices)
		if v[0] > v[1]:
			(print "Male")
		else:
			(print "Female")

Если совсем не получается запрограммировать нейронную сеть самостоятельно, обратитесь к туториалу тут: https://github.com/divamgupta/lstm-gender-predictor